In [35]:
# Imports
import pandas as pd
import numpy as np
import io
import pytz
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
import yfinance as yf
import pandas_ta as ta

## Step 1: Create the Custom Trading Algorithm


In [36]:
#import the S&P 500 data

df = pd.read_csv(
    Path("../Resources/spy.csv"),
    index_col = 'Date',
    infer_datetime_format=True,
    parse_dates=True
)

df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2021-04-14 15:00:00+00:00,63200.328125,63143.218750,63459.949219,560734208,62603.304688,352.438978,-227.296078,579.735056,47.549614,30.005740,45.569075,-1.0,0.0,-1.0,0,NaN,-15.563335,NaN
2021-04-14 16:00:00+00:00,63636.906250,63171.070312,63826.519531,0,62987.664062,316.196716,-210.830672,527.027388,52.571302,29.383467,33.556675,-1.0,0.0,-1.0,0,0.0,-4.173208,0.0
2021-04-14 17:00:00+00:00,62808.371094,63655.718750,63854.359375,0,62728.757812,218.104428,-247.138368,465.242796,43.967897,27.077575,28.822261,-1.0,0.0,-1.0,0,0.0,-1.744686,0.0
2021-04-14 18:00:00+00:00,62198.378906,62545.937500,62730.660156,623951872,61868.546875,90.105629,-300.109733,390.215362,38.918127,21.942582,26.134541,-1.0,0.0,-1.0,0,0.0,-4.191959,0.0
2021-04-14 19:00:00+00:00,62244.957031,62217.371094,62557.453125,1172676608,61554.796875,-7.489541,-318.163923,310.674382,39.489616,13.650148,20.890102,-1.0,0.0,-1.0,0,0.0,-7.239954,0.0


In [37]:
# create a copy of the dataframe
custom_df = df.copy()

custom_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2021-04-14 15:00:00+00:00,63200.328125,63143.218750,63459.949219,560734208,62603.304688,352.438978,-227.296078,579.735056,47.549614,30.005740,45.569075,-1.0,0.0,-1.0,0,NaN,-15.563335,NaN
2021-04-14 16:00:00+00:00,63636.906250,63171.070312,63826.519531,0,62987.664062,316.196716,-210.830672,527.027388,52.571302,29.383467,33.556675,-1.0,0.0,-1.0,0,0.0,-4.173208,0.0
2021-04-14 17:00:00+00:00,62808.371094,63655.718750,63854.359375,0,62728.757812,218.104428,-247.138368,465.242796,43.967897,27.077575,28.822261,-1.0,0.0,-1.0,0,0.0,-1.744686,0.0
2021-04-14 18:00:00+00:00,62198.378906,62545.937500,62730.660156,623951872,61868.546875,90.105629,-300.109733,390.215362,38.918127,21.942582,26.134541,-1.0,0.0,-1.0,0,0.0,-4.191959,0.0
2021-04-14 19:00:00+00:00,62244.957031,62217.371094,62557.453125,1172676608,61554.796875,-7.489541,-318.163923,310.674382,39.489616,13.650148,20.890102,-1.0,0.0,-1.0,0,0.0,-7.239954,0.0


In [38]:
# Calculate stochastic values using the pandas_ta library
custom_df.ta.stoch(close="Close", append=True)

,STOCHk_14_3_3,STOCHd_14_3_3
Date,,
2021-04-15 05:00:00+00:00,NaN,NaN
2021-04-15 06:00:00+00:00,NaN,NaN
2021-04-15 07:00:00+00:00,66.678105,NaN
2021-04-15 08:00:00+00:00,63.435079,NaN
2021-04-15 09:00:00+00:00,62.742926,64.285370
...,...,...
2022-04-13 02:00:00+00:00,48.301882,48.923804
2022-04-13 03:00:00+00:00,55.028137,51.088304
2022-04-13 04:00:00+00:00,66.498858,56.609626


In [39]:
# Calculate MACD values using the pandas_ta library
custom_df.ta.macd(close='Close',  fast=12, slow=26, signal=9, append=True)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,
2021-04-14 15:00:00+00:00,NaN,NaN,NaN
2021-04-14 16:00:00+00:00,NaN,NaN,NaN
2021-04-14 17:00:00+00:00,NaN,NaN,NaN
2021-04-14 18:00:00+00:00,NaN,NaN,NaN
2021-04-14 19:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2022-04-13 02:00:00+00:00,-152.940047,51.444522,-204.384569
2022-04-13 03:00:00+00:00,-129.417894,59.973340,-189.391234
2022-04-13 04:00:00+00:00,-109.789471,63.681410,-173.470881


In [40]:
#drop NaN values

custom_df = custom_df.dropna()

custom_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,STOCHd_14_3_3,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit
Date,,,,,,,,,,,,,,,,,,
2021-04-16 02:00:00+00:00,62890.617188,63178.492188,63293.457031,413024256,62890.617188,90.187448,18.578051,71.609397,46.249224,54.913311,67.146923,1.0,0.0,-1.0,0,0.0,-12.233611,0.0
2021-04-16 03:00:00+00:00,63060.675781,62897.472656,63242.539062,645509120,62867.000000,76.206577,3.677745,72.528833,49.448572,42.054497,55.484842,1.0,0.0,-1.0,0,0.0,-13.430345,0.0
2021-04-16 04:00:00+00:00,61900.785156,63024.421875,63024.421875,137613312,61900.785156,-28.142434,-80.537014,52.394579,34.406195,22.390194,39.786001,-1.0,0.0,-1.0,0,-2.0,-17.395807,0.0
2021-04-16 05:00:00+00:00,61948.710938,61817.359375,62318.277344,1772748800,61695.523438,-105.753508,-126.518470,20.764962,35.282233,16.410575,26.951755,-1.0,0.0,-1.0,0,0.0,-10.541180,0.0
2021-04-16 06:00:00+00:00,61562.843750,61943.515625,61943.515625,1777618944,61289.355469,-196.136164,-173.520901,-22.615263,31.620533,7.569399,15.456722,-1.0,0.0,-1.0,0,0.0,-7.887324,0.0


In [41]:
custom_df["k-d"] = custom_df["STOCHk_14_3_3"] - custom_df["STOCHd_14_3_3"]
custom_df["hist_adj"] = custom_df["MACDh_12_26_9"].diff()

C:\Users\nmend\AppData\Local\Temp/ipykernel_7028/1596102957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom_df["k-d"] = custom_df["STOCHk_14_3_3"] - custom_df["STOCHd_14_3_3"]
C:\Users\nmend\AppData\Local\Temp/ipykernel_7028/1596102957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom_df["hist_adj"] = custom_df["MACDh_12_26_9"].diff()


In [42]:
#Create the Stochastic signal

custom_df["Stoch_Signal"]=0.0

custom_df["Stoch_Signal"] = np.where(
    custom_df["k-d"] > 0, 1.0, 0.0
)


#Create the MACD signal

custom_df["MACD_Signal"]=0.0

custom_df["MACD_Signal"] = np.where(
    custom_df["hist_adj"] > 0, 1.0, 0.0

)
custom_df["Signal"] = 0

for index, row in custom_df.iterrows():

    if (row["Stoch_Signal"] == 1 and row["MACD_Signal"] == 1):
        custom_df.at[index, "Signal"] = 1
    else:
        custom_df.at[index, "Signal"] = 0    

custom_df.head()


C:\Users\nmend\AppData\Local\Temp/ipykernel_7028/1093089296.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom_df["Stoch_Signal"]=0.0
C:\Users\nmend\AppData\Local\Temp/ipykernel_7028/1093089296.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom_df["Stoch_Signal"] = np.where(
C:\Users\nmend\AppData\Local\Temp/ipykernel_7028/1093089296.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,MACD_Signal,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit,k-d,hist_adj,Signal
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-16 02:00:00+00:00,62890.617188,63178.492188,63293.457031,413024256,62890.617188,90.187448,18.578051,71.609397,46.249224,54.913311,...,0.0,0.0,0.0,0,0.0,-12.233611,0.0,-12.233611,NaN,0
2021-04-16 03:00:00+00:00,63060.675781,62897.472656,63242.539062,645509120,62867.000000,76.206577,3.677745,72.528833,49.448572,42.054497,...,0.0,0.0,0.0,0,0.0,-13.430345,0.0,-13.430345,-14.900307,0
2021-04-16 04:00:00+00:00,61900.785156,63024.421875,63024.421875,137613312,61900.785156,-28.142434,-80.537014,52.394579,34.406195,22.390194,...,0.0,0.0,0.0,0,-2.0,-17.395807,0.0,-17.395807,-84.214758,0
2021-04-16 05:00:00+00:00,61948.710938,61817.359375,62318.277344,1772748800,61695.523438,-105.753508,-126.518470,20.764962,35.282233,16.410575,...,0.0,0.0,0.0,0,0.0,-10.541180,0.0,-10.541180,-45.981456,0
2021-04-16 06:00:00+00:00,61562.843750,61943.515625,61943.515625,1777618944,61289.355469,-196.136164,-173.520901,-22.615263,31.620533,7.569399,...,0.0,0.0,0.0,0,0.0,-7.887324,0.0,-7.887324,-47.002431,0


In [43]:
#calculate entry/exit points, 1 or -1
custom_df["Entry/Exit"] = custom_df["Signal"].diff()

#review the dataframe
custom_df.tail(10)

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,RSI_Signal,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit,k-d,hist_adj,Signal,Entry/Exit
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-12 21:00:00+00:00,39727.839844,39558.058594,39760.738281,0,39555.507812,-253.556285,-6.765754,-246.790531,40.890082,13.853967,...,0.0,1.0,0,0.0,2.836205,2.0,2.836205,2.846876,1,1.0
2022-04-12 22:00:00+00:00,39828.750000,39724.785156,39828.750000,0,39637.609375,-241.711684,4.063077,-245.774762,43.150127,25.773453,...,0.0,1.0,0,2.0,10.763385,0.0,10.763385,10.828831,1,0.0
2022-04-12 23:00:00+00:00,40136.175781,39842.210938,40179.984375,232796160,39842.210938,-205.153184,32.497262,-237.650446,49.486676,41.364560,...,0.0,1.0,0,0.0,14.367233,0.0,14.367233,28.434185,1,0.0
2022-04-13 00:00:00+00:00,39991.968750,40111.546875,40176.296875,0,39990.292969,-185.676251,41.579356,-227.255607,46.848810,48.534639,...,0.0,1.0,0,0.0,9.977088,0.0,9.977088,9.082094,1,0.0
2022-04-13 01:00:00+00:00,39880.332031,39979.210938,39979.210938,0,39712.746094,-177.206070,40.039630,-217.245700,44.855453,49.934892,...,0.0,1.0,0,0.0,3.323528,0.0,3.323528,-1.539727,0,-1.0
2022-04-13 02:00:00+00:00,40076.019531,39900.718750,40116.257812,94822400,39790.132812,-152.940047,51.444522,-204.384569,48.955389,48.301882,...,0.0,0.0,0,0.0,-0.621922,-2.0,-0.621922,11.404892,0,0.0
2022-04-13 03:00:00+00:00,40110.710938,40065.468750,40233.812500,19312640,40057.527344,-129.417894,59.973340,-189.391234,49.669797,55.028137,...,0.0,1.0,0,0.0,3.939834,2.0,3.939834,8.528818,1,1.0
2022-04-13 04:00:00+00:00,40107.257812,40110.140625,40125.730469,0,39983.550781,-109.789471,63.681410,-173.470881,49.595390,66.498858,...,0.0,1.0,0,0.0,9.889232,0.0,9.889232,3.708070,1,0.0
2022-04-13 05:00:00+00:00,40135.281250,40136.691406,40153.757812,17272832,40110.160156,-90.924428,66.037163,-156.961591,50.246772,74.238791,...,0.0,1.0,0,0.0,8.983529,0.0,8.983529,2.355753,1,0.0


In [44]:
#set the initial_capital to 100000

initial_capital = 100000

In [45]:
# set the share size to 100
share_size = 100

In [46]:
#create a column named "Position" by multiplying the share_size by the signal
# buy a position when the MACD signal = 1 (MACD histogram adj shows positive value and the stochastic is also showing bullish sentiment.)
# sell a position when the MACD signal = -1 (MACD histogram shows negative value and the stochastic is also showing bearish sentiment.)

custom_df["Position"] = share_size * custom_df["Signal"]

#review the dataframe
custom_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,Stoch_Signal,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit,k-d,hist_adj,Signal,Entry/Exit,Position
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-16 02:00:00+00:00,62890.617188,63178.492188,63293.457031,413024256,62890.617188,90.187448,18.578051,71.609397,46.249224,54.913311,...,0.0,0,0.0,-12.233611,0.0,-12.233611,NaN,0,NaN,0
2021-04-16 03:00:00+00:00,63060.675781,62897.472656,63242.539062,645509120,62867.000000,76.206577,3.677745,72.528833,49.448572,42.054497,...,0.0,0,0.0,-13.430345,0.0,-13.430345,-14.900307,0,0.0,0
2021-04-16 04:00:00+00:00,61900.785156,63024.421875,63024.421875,137613312,61900.785156,-28.142434,-80.537014,52.394579,34.406195,22.390194,...,0.0,0,-2.0,-17.395807,0.0,-17.395807,-84.214758,0,0.0,0
2021-04-16 05:00:00+00:00,61948.710938,61817.359375,62318.277344,1772748800,61695.523438,-105.753508,-126.518470,20.764962,35.282233,16.410575,...,0.0,0,0.0,-10.541180,0.0,-10.541180,-45.981456,0,0.0,0
2021-04-16 06:00:00+00:00,61562.843750,61943.515625,61943.515625,1777618944,61289.355469,-196.136164,-173.520901,-22.615263,31.620533,7.569399,...,0.0,0,0.0,-7.887324,0.0,-7.887324,-47.002431,0,0.0,0


In [47]:
#find the points in time where a position is purchased or sold
custom_df["Entry/Exit Position"] = custom_df["Position"].diff()

#review the dataframe
custom_df.head()


,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,custom_signal,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit,k-d,hist_adj,Signal,Entry/Exit,Position,Entry/Exit Position
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-16 02:00:00+00:00,62890.617188,63178.492188,63293.457031,413024256,62890.617188,90.187448,18.578051,71.609397,46.249224,54.913311,...,0,0.0,-12.233611,0.0,-12.233611,NaN,0,NaN,0,NaN
2021-04-16 03:00:00+00:00,63060.675781,62897.472656,63242.539062,645509120,62867.000000,76.206577,3.677745,72.528833,49.448572,42.054497,...,0,0.0,-13.430345,0.0,-13.430345,-14.900307,0,0.0,0,0.0
2021-04-16 04:00:00+00:00,61900.785156,63024.421875,63024.421875,137613312,61900.785156,-28.142434,-80.537014,52.394579,34.406195,22.390194,...,0,-2.0,-17.395807,0.0,-17.395807,-84.214758,0,0.0,0,0.0
2021-04-16 05:00:00+00:00,61948.710938,61817.359375,62318.277344,1772748800,61695.523438,-105.753508,-126.518470,20.764962,35.282233,16.410575,...,0,0.0,-10.541180,0.0,-10.541180,-45.981456,0,0.0,0,0.0
2021-04-16 06:00:00+00:00,61562.843750,61943.515625,61943.515625,1777618944,61289.355469,-196.136164,-173.520901,-22.615263,31.620533,7.569399,...,0,0.0,-7.887324,0.0,-7.887324,-47.002431,0,0.0,0,0.0


In [48]:
custom_df = custom_df.dropna()

In [49]:
#create a Portfolio Holdings column by multiplying the Close price by the Position

custom_df["Portfolio Holdings"] = custom_df["Close"] * custom_df["Position"]

#review the dataframe
custom_df.head()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,MACD_Entry/Exit,stoch_diff,Stoch_Entry/Exit,k-d,hist_adj,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-16 03:00:00+00:00,63060.675781,62897.472656,63242.539062,645509120,62867.000000,76.206577,3.677745,72.528833,49.448572,42.054497,...,0.0,-13.430345,0.0,-13.430345,-14.900307,0,0.0,0,0.0,0.0
2021-04-16 04:00:00+00:00,61900.785156,63024.421875,63024.421875,137613312,61900.785156,-28.142434,-80.537014,52.394579,34.406195,22.390194,...,-2.0,-17.395807,0.0,-17.395807,-84.214758,0,0.0,0,0.0,0.0
2021-04-16 05:00:00+00:00,61948.710938,61817.359375,62318.277344,1772748800,61695.523438,-105.753508,-126.518470,20.764962,35.282233,16.410575,...,0.0,-10.541180,0.0,-10.541180,-45.981456,0,0.0,0,0.0,0.0
2021-04-16 06:00:00+00:00,61562.843750,61943.515625,61943.515625,1777618944,61289.355469,-196.136164,-173.520901,-22.615263,31.620533,7.569399,...,0.0,-7.887324,0.0,-7.887324,-47.002431,0,0.0,0,0.0,0.0
2021-04-16 07:00:00+00:00,61626.218750,61558.093750,61724.355469,550719488,61278.593750,-259.658061,-189.634238,-70.023823,32.853119,12.125885,...,0.0,0.090599,2.0,0.090599,-16.113337,0,0.0,0,0.0,0.0


In [50]:
#To calculate Portfolio Cash, subtrace the cumulative sum of the trade cost/proceeds from the initial_capital
#The trade cost proceeds are calculated by multiplying the Close price by the Entry/Exit Position

custom_df["Portfolio Cash"] = initial_capital - (custom_df["Close"] * custom_df["Entry/Exit Position"]).cumsum()

#review the dataframe
custom_df.tail(10)

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,stoch_diff,Stoch_Entry/Exit,k-d,hist_adj,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-12 21:00:00+00:00,39727.839844,39558.058594,39760.738281,0,39555.507812,-253.556285,-6.765754,-246.790531,40.890082,13.853967,...,2.836205,2.0,2.836205,2.846876,1,1.0,1,1.0,39727.839844,38892.244141
2022-04-12 22:00:00+00:00,39828.750000,39724.785156,39828.750000,0,39637.609375,-241.711684,4.063077,-245.774762,43.150127,25.773453,...,10.763385,0.0,10.763385,10.828831,1,0.0,1,0.0,39828.750000,38892.244141
2022-04-12 23:00:00+00:00,40136.175781,39842.210938,40179.984375,232796160,39842.210938,-205.153184,32.497262,-237.650446,49.486676,41.364560,...,14.367233,0.0,14.367233,28.434185,1,0.0,1,0.0,40136.175781,38892.244141
2022-04-13 00:00:00+00:00,39991.968750,40111.546875,40176.296875,0,39990.292969,-185.676251,41.579356,-227.255607,46.848810,48.534639,...,9.977088,0.0,9.977088,9.082094,1,0.0,1,0.0,39991.968750,38892.244141
2022-04-13 01:00:00+00:00,39880.332031,39979.210938,39979.210938,0,39712.746094,-177.206070,40.039630,-217.245700,44.855453,49.934892,...,3.323528,0.0,3.323528,-1.539727,0,-1.0,0,-1.0,0.000000,78772.576172
2022-04-13 02:00:00+00:00,40076.019531,39900.718750,40116.257812,94822400,39790.132812,-152.940047,51.444522,-204.384569,48.955389,48.301882,...,-0.621922,-2.0,-0.621922,11.404892,0,0.0,0,0.0,0.000000,78772.576172
2022-04-13 03:00:00+00:00,40110.710938,40065.468750,40233.812500,19312640,40057.527344,-129.417894,59.973340,-189.391234,49.669797,55.028137,...,3.939834,2.0,3.939834,8.528818,1,1.0,1,1.0,40110.710938,38661.865234
2022-04-13 04:00:00+00:00,40107.257812,40110.140625,40125.730469,0,39983.550781,-109.789471,63.681410,-173.470881,49.595390,66.498858,...,9.889232,0.0,9.889232,3.708070,1,0.0,1,0.0,40107.257812,38661.865234
2022-04-13 05:00:00+00:00,40135.281250,40136.691406,40153.757812,17272832,40110.160156,-90.924428,66.037163,-156.961591,50.246772,74.238791,...,8.983529,0.0,8.983529,2.355753,1,0.0,1,0.0,40135.281250,38661.865234


In [51]:
#calculate the Portfolo Total by adding Portfolio Cash and Portfolio Holdings
custom_df["Portfolio Total"] = custom_df["Portfolio Cash"] + custom_df["Portfolio Holdings"]

#review the dataframe
custom_df.tail()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,Stoch_Entry/Exit,k-d,hist_adj,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-13 02:00:00+00:00,40076.019531,39900.718750,40116.257812,94822400,39790.132812,-152.940047,51.444522,-204.384569,48.955389,48.301882,...,-2.0,-0.621922,11.404892,0,0.0,0,0.0,0.000000,78772.576172,78772.576172
2022-04-13 03:00:00+00:00,40110.710938,40065.468750,40233.812500,19312640,40057.527344,-129.417894,59.973340,-189.391234,49.669797,55.028137,...,2.0,3.939834,8.528818,1,1.0,1,1.0,40110.710938,38661.865234,78772.576172
2022-04-13 04:00:00+00:00,40107.257812,40110.140625,40125.730469,0,39983.550781,-109.789471,63.681410,-173.470881,49.595390,66.498858,...,0.0,9.889232,3.708070,1,0.0,1,0.0,40107.257812,38661.865234,78769.123047
2022-04-13 05:00:00+00:00,40135.281250,40136.691406,40153.757812,17272832,40110.160156,-90.924428,66.037163,-156.961591,50.246772,74.238791,...,0.0,8.983529,2.355753,1,0.0,1,0.0,40135.281250,38661.865234,78797.146484
2022-04-13 05:37:00+00:00,40169.285156,40169.285156,40169.285156,0,40169.285156,-72.395385,67.652965,-140.048349,51.073012,81.947600,...,0.0,7.719183,1.615802,1,0.0,1,0.0,40169.285156,38661.865234,78831.150391


In [52]:
#calculate the Portfolio Period Returns based on the Portfolio Total
custom_df["Portfolio Period Returns"] = custom_df["Portfolio Total"].pct_change()

#review the dataframe
custom_df.tail()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,k-d,hist_adj,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-13 02:00:00+00:00,40076.019531,39900.718750,40116.257812,94822400,39790.132812,-152.940047,51.444522,-204.384569,48.955389,48.301882,...,-0.621922,11.404892,0,0.0,0,0.0,0.000000,78772.576172,78772.576172,0.000000
2022-04-13 03:00:00+00:00,40110.710938,40065.468750,40233.812500,19312640,40057.527344,-129.417894,59.973340,-189.391234,49.669797,55.028137,...,3.939834,8.528818,1,1.0,1,1.0,40110.710938,38661.865234,78772.576172,0.000000
2022-04-13 04:00:00+00:00,40107.257812,40110.140625,40125.730469,0,39983.550781,-109.789471,63.681410,-173.470881,49.595390,66.498858,...,9.889232,3.708070,1,0.0,1,0.0,40107.257812,38661.865234,78769.123047,-0.000044
2022-04-13 05:00:00+00:00,40135.281250,40136.691406,40153.757812,17272832,40110.160156,-90.924428,66.037163,-156.961591,50.246772,74.238791,...,8.983529,2.355753,1,0.0,1,0.0,40135.281250,38661.865234,78797.146484,0.000356
2022-04-13 05:37:00+00:00,40169.285156,40169.285156,40169.285156,0,40169.285156,-72.395385,67.652965,-140.048349,51.073012,81.947600,...,7.719183,1.615802,1,0.0,1,0.0,40169.285156,38661.865234,78831.150391,0.000432


In [53]:
#Calculate the Portfolio Cumulative Returns based on the Portfolio Daily Returns
custom_df["Portfolio Cumulative Returns"] = (1 + custom_df["Portfolio Period Returns"]).cumprod() - 1

#review the dataframe
custom_df.tail()

,Close,Open,High,Volume,Low,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,STOCHk_14_3_3,...,hist_adj,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Period Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-13 02:00:00+00:00,40076.019531,39900.718750,40116.257812,94822400,39790.132812,-152.940047,51.444522,-204.384569,48.955389,48.301882,...,11.404892,0,0.0,0,0.0,0.000000,78772.576172,78772.576172,0.000000,-0.212274
2022-04-13 03:00:00+00:00,40110.710938,40065.468750,40233.812500,19312640,40057.527344,-129.417894,59.973340,-189.391234,49.669797,55.028137,...,8.528818,1,1.0,1,1.0,40110.710938,38661.865234,78772.576172,0.000000,-0.212274
2022-04-13 04:00:00+00:00,40107.257812,40110.140625,40125.730469,0,39983.550781,-109.789471,63.681410,-173.470881,49.595390,66.498858,...,3.708070,1,0.0,1,0.0,40107.257812,38661.865234,78769.123047,-0.000044,-0.212309
2022-04-13 05:00:00+00:00,40135.281250,40136.691406,40153.757812,17272832,40110.160156,-90.924428,66.037163,-156.961591,50.246772,74.238791,...,2.355753,1,0.0,1,0.0,40135.281250,38661.865234,78797.146484,0.000356,-0.212029
2022-04-13 05:37:00+00:00,40169.285156,40169.285156,40169.285156,0,40169.285156,-72.395385,67.652965,-140.048349,51.073012,81.947600,...,1.615802,1,0.0,1,0.0,40169.285156,38661.865234,78831.150391,0.000432,-0.211688


## Visualize the Custom Algorithm

In [54]:
#visualize entry positions relative to close price

entry = custom_df[custom_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the exit positions relative to close price
exit = custom_df[custom_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Price in $",
    width = 1400,
    height = 600)

#visualize the close price for investment
security_close = custom_df[['Close']].hvplot(
    line_color = "lightblue",
    ylabel = "Price in $",
    width = 1400,
    height = 600)

entry_exit_plot = security_close * entry * exit
entry_exit_plot.opts(title = "Custom Trading Algorithm Entry/Exits")

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [55]:
#visualize entry positions relative to close price

entry = custom_df[custom_df["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color = 'purple',
    marker = '^',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

#visualize the exit positions relative to close price
exit = custom_df[custom_df["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color = 'orange',
    marker = 'v',
    legend = False,
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

#visualize the Portfolio Total for investment
total_portfolio_value = custom_df[['Portfolio Total']].hvplot(
    line_color = "lightblue",
    ylabel = "Total Portfolio Value",
    width = 1400,
    height = 600)

portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title = "Custom Trading Algorithm - Total Portfolio Value",
    yformatter="%.0f"
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

## Evaluate the portfolio metrics of the Stochastic Algorithm

In [56]:
#create a new DataFrame to evaluate the performance of the algorithm

# create a list for the column name

columns = ["Backtest"]

metrics = [

    "Annualized Return",
    "Cumulative Returns",
    "Annual Volatility",
    "Sharpe Ratio"
    ]
portfolio_evaluation_df = pd.DataFrame(index = metrics, columns=columns)

portfolio_evaluation_df.head()

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [57]:
# calculate the annualized return

portfolio_evaluation_df.loc["Annualized Return"] = (

    custom_df["Portfolio Period Returns"].mean() * 252 * 7

)

portfolio_evaluation_df

,Backtest
Annualized Return,-0.043017
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN


In [58]:
# calculate the cumulative return

portfolio_evaluation_df.loc["Cumulative Returns"] = (

    custom_df["Portfolio Cumulative Returns"][-1]
)

portfolio_evaluation_df

,Backtest
Annualized Return,-0.043017
Cumulative Returns,-0.211688
Annual Volatility,NaN
Sharpe Ratio,NaN


In [59]:
# Calculate the annual volatility

portfolio_evaluation_df.loc["Annual Volatility"] = (

    custom_df["Portfolio Period Returns"].std() * np.sqrt(252*7)

)

portfolio_evaluation_df

,Backtest
Annualized Return,-0.043017
Cumulative Returns,-0.211688
Annual Volatility,0.113391
Sharpe Ratio,NaN


In [60]:
# Calculate the Sharpe Ratio

portfolio_evaluation_df.loc["Sharpe Ratio"] = (

    custom_df["Portfolio Period Returns"].mean() * 252) / (

    custom_df["Portfolio Period Returns"].std() * np.sqrt(252)

)

portfolio_evaluation_df

,Backtest
Annualized Return,-0.043017
Cumulative Returns,-0.211688
Annual Volatility,0.113391
Sharpe Ratio,-0.143386


## Trade-Level Analytics

In [61]:
custom_df["value"] =  custom_df["Entry/Exit Position"] * custom_df["Close"]

In [62]:
custom_df["Entry/Exit Position"].value_counts()

 0.0    6495
 1.0     996
-1.0     995
Name: Entry/Exit Position, dtype: int64

In [70]:
# create a new dataframe to evaluate trade-level Risk/Reward Metrics

trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Value',
        'Exit Portfolio Value',
        'Profit/Loss']
)


In [71]:
#initialize the iterative values
entry_date = ""
exit_date = ""
entry_portfolio_value = 0.0
exit_portfolio_value = 0.0
share_size = 0.0
entry_share_price = 0.0
exit_share_price = 0.0


In [72]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in custom_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_value = row['value']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_value = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_value - entry_portfolio_value
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'BTC',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Value': entry_portfolio_value,
                'Exit Portfolio Value': exit_portfolio_value,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Value,Exit Portfolio Value,Profit/Loss
0,BTC,2021-04-16 11:00:00+00:00,2021-04-16 22:00:00+00:00,1.0,60940.941406,61726.996094,60940.941406,61726.996094,786.054688
1,BTC,2021-04-17 02:00:00+00:00,2021-04-17 04:00:00+00:00,1.0,62205.984375,62282.492188,62205.984375,62282.492188,76.507812
2,BTC,2021-04-17 08:00:00+00:00,2021-04-17 09:00:00+00:00,1.0,62396.230469,61784.972656,62396.230469,61784.972656,-611.257812
3,BTC,2021-04-17 18:00:00+00:00,2021-04-18 00:00:00+00:00,1.0,60972.386719,60285.296875,60972.386719,60285.296875,-687.089844
4,BTC,2021-04-18 05:00:00+00:00,2021-04-18 08:00:00+00:00,1.0,56848.148438,55048.769531,56848.148438,55048.769531,-1799.378906
...,...,...,...,...,...,...,...,...,...
990,BTC,2022-04-11 05:00:00+00:00,2022-04-11 08:00:00+00:00,1.0,42283.246094,42231.617188,42283.246094,42231.617188,-51.628906
991,BTC,2022-04-11 18:00:00+00:00,2022-04-11 19:00:00+00:00,1.0,40507.578125,40075.941406,40507.578125,40075.941406,-431.636719
992,BTC,2022-04-11 22:00:00+00:00,2022-04-12 01:00:00+00:00,1.0,39855.812500,39555.921875,39855.812500,39555.921875,-299.890625
993,BTC,2022-04-12 03:00:00+00:00,2022-04-12 13:00:00+00:00,1.0,39726.667969,40454.300781,39726.667969,40454.300781,727.632812


In [73]:
#determine a winning vs. losing trade

win = 0
loss = 0
max_win = trade_evaluation_df["Profit/Loss"].max()
max_loss = trade_evaluation_df["Profit/Loss"].min()

for pnl in trade_evaluation_df["Profit/Loss"]:
    if pnl > 0:
        win +=1
    else:
        loss +=1

winrate = win / (win + loss)

print(f"There were {win} winning trades and {loss} losing trades giving us a winrate of {winrate})")
print(f"The largest gain made was ${max_win} and the largest loss was ${max_loss}")

There were 341 winning trades and 654 losing trades giving us a winrate of 0.342713567839196)
The largest gain made was $3717.39453125 and the largest loss was $-3092.05078125


In [67]:
#let's see how much money our system made in total:

total_profit_loss = trade_evaluation_df["Profit/Loss"].sum()

print(f"Using our algorithm made ${total_profit_loss}!")

Using our algorithm made $-21227.423828125!


In [68]:
#export the dataframe to a csv file
#custom_df.to_csv("../Resources/custom_new.csv")